<a href="https://colab.research.google.com/github/AkshitSingh11/Lunar-Lander/blob/main/Lunar_Lander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing the required packages and importing the libraries

## Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


## Importing the libraries

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple
import gymnasium as gym

# Building the AI

**Observational Space** :-<br> The state is an 8-dimensional vector: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.
<br>
**Action Space** :-<br> There are four discrete actions available:<br>
0: do nothing<br>
1: fire left orientation engine<br>
2: fire main engine<br>
3: fire right orientation engine<br>
**Rewards**:-<br>
For each step, the reward:<br>
1) is increased/decreased the closer/further the lander is to the landing pad.<br>
2) is increased/decreased the slower/faster the lander is moving.<br>
3) is decreased the more the lander is tilted (angle not horizontal).<br>
4) is increased by 10 points for each leg that is in contact with the ground.<br>
5) is decreased by 0.03 points each frame a side engine is firing.<br>
6) is decreased by 0.3 points each frame the main engine is firing.<br>



## Creating the architecture of the Neural Network


In [3]:
class Network(nn.Module):

  def __init__(self, state_size, action_size, seed =42):
    super(Network, self).__init__()
    self.seed = torch.manual_seed(seed)
    self.fc1 = nn.Linear(state_size, 64)
    self.fc2 = nn.Linear(64, 64)
    self.fc3 = nn.Linear(64, action_size)

  def forward(self, state):
    x = self.fc1(state)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    return self.fc3(x)

# Training the AI

## Setting up the Environment

In [4]:
env = gym.make('LunarLander-v3')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n

In [5]:
print('State shape: ',state_shape)
print('State size: ',state_size)
print('Number of actions: ',number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


## Initializing the hyperparameters

In [6]:
learning_rate = 5e-4
minibatch_size = 100       # Over this many experiences at a time, our agent will learn
discount_factor = 0.99
replay_buffer_size = int(1e5)    # Size of memory to store experiences
interpolation_parameter = 1e-3

##Implementing Experience Replay

In [7]:
class ReplayMemory(object):

  def __init__(self,capacity):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.capacity = capacity
    self.memory = []      # Store experiences

  # Function to store experience
  def push(self, event):
    self.memory.append(event)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  # Function to return a sample of a certain size
  def sample(self, batch_size):
    experiences = random.sample(self.memory, k=batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)      # If the ander has landed or not
    return states, next_states, actions, rewards, dones

## Implementing the DQN class

In [8]:
class Agent():

  def __init__(self, state_size,action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.state_size = state_size
    self.action_size = action_size
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    self.target_qnetwork = Network(state_size, action_size).to(self.device)    # to stabilise learning process
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr= learning_rate)
    self.memory = ReplayMemory(replay_buffer_size)
    self.t_step = 0     # To keep count of step at which we will update the network parameters

  def step(self, state,action,reward,next_state,done):
    self.memory.push((state,action,reward,next_state,done))
    self.t_step = (self.t_step +1)% 4      #Learn after every 4 steps
    if self.t_step ==0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  # Using epsilon greedy action selection
  def act(self, state, epsilon =0.):
    state  = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    self.local_qnetwork.eval()
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()
    if random.random() >epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn(self, experiences, discount_factor):
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
    q_expected = self.local_qnetwork(states).gather(1,actions)
    loss = F.mse_loss(q_expected, q_targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  # Update target parameters
  def soft_update(self,local_model, target_model,iterpolation_parameter):
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)

## Initializing the DQN agent

In [9]:
agent = Agent(state_size, number_actions)

## Training the DQN agent

In [10]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value = 1.0
epsilon_ending_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen=100)

for episode in range(1,number_episodes+1):
  state, _ = env.reset()
  score =0
  for t in range(maximum_number_timesteps_per_episode):
    action = agent.act(state, epsilon)
    next_state, reward, done, _, _ = env.step(action)
    agent.step(state, action, reward, next_state, done)
    state = next_state
    score += reward
    if done:
      break
  scores_on_100_episodes.append(score)
  epsilon = max(epsilon_ending_value, epsilon * epsilon_decay_value)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode %100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode-100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break

Episode 100	Average Score: -162.78
Episode 200	Average Score: -98.70
Episode 300	Average Score: -53.64
Episode 400	Average Score: 32.93
Episode 500	Average Score: 130.55
Episode 600	Average Score: 193.63
Episode 700	Average Score: 168.23
Episode 764	Average Score: 200.00
Environment solved in 664 episodes!	Average Score: 200.00


# Visualizing the results

In [15]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()